# Project: Predict Future Sales
#### Notebook 2 of 4: building hierarchical time series and top level forecasting

In this notebook, we mainly did the following:
- Built hierarchical time series for each shop
- Forecasting all top level hierachy time series using prophet


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib
from matplotlib import pyplot as plt
from textwrap import wrap


pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'

import pickle
import time
from tqdm import tqdm

from itertools import product

#import googletrans
#from googletrans import Translator
#to avoid instantiate too many translator, we created one instance for the whole notebook
#translator = Translator()


from prophet import Prophet
import itertools
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import holidays


import hts.functions
import collections
from hts import HTSRegressor
from hts.hierarchy import HierarchyTree


import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

# define functions

In [2]:
def get_best_hyperparams(data):   
    
    default_best_params = {'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.5}
    
    first_cutoff_date = pd.to_datetime('2014-02-1')
    first_nonzero_date = data['y'].ne(0).idxmax()
    
    # return default_best_params if 
    # 1. all values are zero
    # 2. too many values are zero
    if (data['y'] == 0).all() or (first_cutoff_date < first_nonzero_date):
        print( first_nonzero_date)
        print("too many zero values, return default best params")
        return default_best_params
    else:
        param_grid = {  
                'changepoint_prior_scale': [0.01,0.2],
                'seasonality_prior_scale': [0.01,0.5 ],
        }

        # Generate all combinations of parameters
        all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
        rmses = []  # Store the RMSEs for each params here

        # Use cross validation to evaluate all parameters
        cutoffs = pd.to_datetime(['2015-8-1', '2015-9-1'])
        #cutoffs = pd.to_datetime(['2015-9-1'])
        
        
        for params in all_params:
            m = Prophet(**params).fit(data)  # Fit model with given params
            df_cv = cross_validation(m, cutoffs=cutoffs, horizon='60 days', parallel="processes")
            df_p = performance_metrics(df_cv, rolling_window=1)
            rmses.append(df_p['rmse'].values[0])

        # Find the best parameters
        tuning_results = pd.DataFrame(all_params)
        tuning_results['rmse'] = rmses

        best_params = all_params[np.argmin(rmses)]
        print(best_params)
        return best_params


In [3]:
def get_best_preds(df_,   #df_heir_train
                   cat_id  #cat_id in oneshop_cat_list 
                  ):
    #get dataframe
    columns = ['ds', cat_id]
    df = df_.loc[:, columns]
    df.columns = ['ds', 'y']
    
    #get best_params
    best_params = get_best_hyperparams(df)
    
    #create model based on best_params
    m = Prophet(changepoint_prior_scale=best_params['changepoint_prior_scale'], 
                seasonality_prior_scale=best_params['seasonality_prior_scale'], 
                yearly_seasonality=True)
    m.add_country_holidays(country_name='RU')
    m.fit(df)

    #predict and return
    future = m.make_future_dataframe(periods=1, freq='MS')
    forecast = m.predict(future)

    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(3))
    #fig1 = m.plot(forecast)

    best_preds = forecast.tail(1).yhat.values[0]
    print('best_preds: ', best_preds)
    return best_preds
    

# import data from pickle file

In [4]:
pickle_dict2 = pickle.load(open('../temp/data2.pkl', 'rb'))
df_basegrid = pickle_dict2['df_basegrid']   #concat df_train_m and df_test


In [5]:
list_shop_id=sorted(df_basegrid['shop_id'].unique())
print(list_shop_id)
print(len(list_shop_id))

[2, 3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 26, 28, 31, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59]
42


# create hierachical time series for all shops

In [4]:
%%time

for i in tqdm(range(len(list_shop_id))):
    ################ get one shop dataframe
    #################################################################
    df_oneshop = df_basegrid[df_basegrid['shop_id'] == list_shop_id[i]]

    ##change values in the following columns so that their combinations are more readable
    df_oneshop['date_block_num'] =df_oneshop['date_block_num'].astype(str)
    df_oneshop['item_category_id'] =df_oneshop['item_category_id'].astype(str)
    df_oneshop['shop_id'] =df_oneshop['shop_id'].astype(str)
    df_oneshop['item_id'] =df_oneshop['item_id'].astype(str)
    df_oneshop['item_category_id'] = 'c' + df_oneshop['item_category_id']
    df_oneshop['shop_id'] = 's' + df_oneshop['shop_id']
    df_oneshop['item_id'] = 'i' + df_oneshop['item_id']



    ############### create hierachical time series for the shop
    ##############################################################
    level_names = ['item_category_id','item_id']
    hierarchy = [['item_category_id']]
    df_hier, sum_mat, sum_mat_labels = hts.functions.get_hierarchichal_df(df_oneshop,
                                                                          level_names=level_names,
                                                                          hierarchy=hierarchy,
                                                                          date_colname='year_month',
                                                                          val_colname='item_cnt_month')
    #fill all null value with 0
    df_hier = df_hier.fillna(0)
    df_hier['ds']= df_hier.index

    old_column_list = df_hier.columns
    new_column_list = [old_column_list[-1], *old_column_list[:-1]]
    df_hier = df_hier[new_column_list]

    
    ################ get the hier_dict
    #################################################################

    #get the lists
    oneshop_cat_list = sorted(df_oneshop['item_category_id'].unique())
    oneshop_item_list = sorted(df_oneshop['item_id'].unique())

    #dictionary to hold all level nodes
    hier_dict = {}

    #get level 1 nodes: category nodes
    level_1_nodes = [str(cat_id) for cat_id in oneshop_cat_list]

    #print(level_1_nodes)
    hier_dict['total'] = level_1_nodes

    #get level 2 nodes
    df_hier_columns = sorted(df_hier.columns)
    for node in level_1_nodes: 
        temp_level_2_nodes = list(filter(lambda cat_item: f'{node}_' in cat_item, df_hier_columns))
        hier_dict[node] = temp_level_2_nodes

    tree = HierarchyTree.from_nodes(hier_dict, df_hier, root='total')
    #print(tree)

    sum_mat, sum_mat_labels = hts.functions.to_sum_mat(tree)



    ###########save neccessary object to pickle_dict, 
    ############################################################################
    pickle_dict_oneshop = dict()
    pickle_dict_oneshop['df_hier']=df_hier   #concat df_train_m and df_test
    pickle_dict_oneshop['tree']=tree 
    pickle_dict_oneshop['sum_mat']=sum_mat
    pickle_dict_oneshop['sum_mat_labels']=sum_mat_labels
    pickle_dict_oneshop['oneshop_cat_list']=oneshop_cat_list
    pickle_dict_oneshop['oneshop_item_list']=oneshop_item_list

    pickle.dump(pickle_dict_oneshop, open(f'../temp/s{list_shop_id[i]}_hier.pkl', 'wb'))




100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [07:28<00:00, 10.67s/it]

Wall time: 7min 28s


# forecasting all hierachy time series using prophet

### shop_id: 36 only have one month data, won't be feed into forcasting

In [6]:
list_shop_id_filtered = list_shop_id.copy()
list_shop_id_filtered.remove(36)
print(list_shop_id_filtered)
print(len(list_shop_id_filtered))

[2, 3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 26, 28, 31, 34, 35, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59]
41


### forecasting each category sales in each shops

In [7]:
%%time
    
for shop_id in tqdm(list_shop_id_filtered):
    
    print(f'\n\nforecasting shop: s{shop_id}')
    
    ####read pickle_dict, which store everything
    #############################################
    pickle_dict_oneshop = pickle.load(open(f'../temp/s{shop_id}_hier.pkl', 'rb'))
    df_hier  = pickle_dict_oneshop['df_hier']
    tree = pickle_dict_oneshop['tree']
    sum_mat = pickle_dict_oneshop['sum_mat']
    sum_mat_labels =pickle_dict_oneshop['sum_mat_labels']
    oneshop_cat_list = pickle_dict_oneshop['oneshop_cat_list']
    oneshop_item_list = pickle_dict_oneshop['oneshop_item_list']


    ################### creat train, validation,and test dataset
    ###########################################################################
    df_hier_train = df_hier.loc[df_hier.index <= '2015-10-01']
    df_hier_test = df_hier.loc[df_hier.index == '2015-11-01']


    
    forecasts = pd.DataFrame(columns = df_hier.columns, index=['fake'])
    
    
    ################### forecasting total sales in this shop
    #################################################################
    print('\ngetting best preds for forecast_total_sales')
    forecasts['total'] = [get_best_preds(df_hier_train, 'total')]

    
  
    ################### forecasting all category sales in this shop
    #################################################################
    #dataframe to hold the forecasts
    for cat in oneshop_cat_list:
        print(f'\ngetting best preds for s{shop_id}_', cat)
        forecasts[cat] = [get_best_preds(df_hier_train, cat)]


    ###########save neccessary object to pickle_dict, 
    ############################################################################
    pickle_dict_oneshop_new = dict()
    pickle_dict_oneshop_new['forecasts']=forecasts
    pickle.dump(pickle_dict_oneshop_new, open(f'../temp/s{shop_id}_forecasts.pkl', 'wb'))



  0%|                                                                                           | 0/41 [00:00<?, ?it/s]



forecasting shop: s2

getting best preds for forecast_total_sales
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds   yhat  yhat_lower  yhat_upper
32 2015-09-01 779.42      697.43      860.33
33 2015-10-01 746.11      660.60      826.48
34 2015-11-01 903.50      818.46      986.13
best_preds:  903.5042053640055

getting best preds for s2_ c0
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for s2_ c11
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.14       -0.22        0.49
33 2015-10-01  0.13       -0.22        0.49
34 2015-11-01  0.20       -0.13        0.54
best_preds:  0.19569609000089122

getting best preds for s2_ c12
{'changepoint_prior_sca

{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 45.31       40.60       50.24
33 2015-10-01 62.51       57.97       67.58
34 2015-11-01 49.49       44.54       54.03
best_preds:  49.487794307098646

getting best preds for s2_ c41
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  2.25        0.62        3.98
33 2015-10-01  3.57        1.93        5.14
34 2015-11-01  4.16        2.47        5.75
best_preds:  4.161186896000797

getting best preds for s2_ c42
2015-05-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  8.38        7.29        9.64
33 2015-10-01  8.59        7.51        9.74
34 2015-11-01  9.84        8.73       11.02
best_preds:  9.841584315476958

getting best preds for s2_ c43
{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.01}
           ds  y

{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  2.63       -0.06        4.90
33 2015-10-01  2.61       -0.09        5.20
34 2015-11-01  2.52        0.01        4.97
best_preds:  2.5193169282489016

getting best preds for s2_ c74
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for s2_ c75
{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  6.35        3.14        9.55
33 2015-10-01  6.10        2.88        9.33
34 2015-11-01  6.29        2.88        9.73
best_preds:  6.294901935196603

getting best preds for s2_ c76
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat

  2%|█▉                                                                              | 1/41 [14:02<9:21:47, 842.68s/it]

...

           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0


forecasting shop: s59

getting best preds for forecast_total_sales
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds     yhat  yhat_lower  yhat_upper
32 2015-09-01   871.35      828.64      917.38
33 2015-10-01   830.14      785.52      872.74
34 2015-11-01 1,218.78    1,174.33    1,262.89
best_preds:  1218.7844846370456

getting best preds for s59_ c0
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for s59_ c11
2015-05-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-0

           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 14.01       10.52       17.49
33 2015-10-01 13.92       10.65       17.64
34 2015-11-01 10.77        7.13       14.37
best_preds:  10.76631688357458

getting best preds for s59_ c40
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 75.88       52.90       99.47
33 2015-10-01 77.41       53.97      100.86
34 2015-11-01 80.90       56.75      104.54
best_preds:  80.89968755897151

getting best preds for s59_ c41
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  7.08        3.79       10.55
33 2015-10-01  7.42        4.11       10.90
34 2015-11-01  7.48        4.09       11.06
best_preds:  7.482486742832844

getting best preds for s59_ c42
2015-06-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  3.91        3.22  

           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 18.21       10.77       25.44
33 2015-10-01 18.26       10.80       25.83
34 2015-11-01 28.65       21.08       36.25
best_preds:  28.649871131047593

getting best preds for s59_ c73
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  2.16       -0.00        4.43
33 2015-10-01  2.18       -0.03        4.40
34 2015-11-01  2.09       -0.16        4.32
best_preds:  2.085178554280344

getting best preds for s59_ c74
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for s59_ c75
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 16.13        8.31       23.30
33 

100%|███████████████████████████████████████████████████████████████████████████████| 41/41 [9:36:49<00:00, 844.13s/it]

           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0
Wall time: 9h 36min 49s


In [23]:
df_hier['c64']

year_month
2013-01-01    4.00
2013-02-01    1.00
2013-03-01    4.00
2013-04-01    3.00
2013-05-01    9.00
2013-06-01    3.00
2013-07-01   13.00
2013-08-01   14.00
2013-09-01    7.00
2013-10-01    4.00
2013-11-01    4.00
2013-12-01   23.00
2014-01-01   10.00
2014-02-01    7.00
2014-03-01    9.00
2014-04-01    8.00
2014-05-01    7.00
2014-06-01   15.00
2014-07-01   15.00
2014-08-01   17.00
2014-09-01   19.00
2014-10-01   12.00
2014-11-01   21.00
2014-12-01   57.00
2015-01-01   38.00
2015-02-01   26.00
2015-03-01   34.00
2015-04-01   19.00
2015-05-01   12.00
2015-06-01   13.00
2015-07-01   21.00
2015-08-01   36.00
2015-09-01   22.00
2015-10-01   25.00
2015-11-01    0.00
Name: c64, dtype: float64